# 📬 Spam Email Classification using Logistic Regression  
*Phân loại Email Spam bằng Hồi quy Logistic*

In this notebook, we will build a simple machine learning model using **Logistic Regression** to classify emails as **Spam** or **Not Spam**.  
Trong notebook này, chúng ta sẽ xây dựng một mô hình học máy đơn giản sử dụng **Hồi quy Logistic** để phân loại email thành **Spam** hoặc **Không phải Spam**.

---

## ✅ Steps | Các bước thực hiện:

1. **Install dependencies**  
   *Cài đặt các thư viện cần thiết*

2. **Load and explore the dataset**  
   *Tải và khám phá bộ dữ liệu*

3. **Preprocess the data** *(feature engineering, scaling)*  
   *Tiền xử lý dữ liệu* *(kỹ thuật đặc trưng, chuẩn hóa)*

4. **Build the Logistic Regression model**  
   *Xây dựng mô hình Logistic Regression*

5. **Evaluate the model's performance**  
   *Đánh giá hiệu suất mô hình*

6. **Make predictions on new data**  
   *Dự đoán với dữ liệu mới*

---

## 🔧 1. Install Dependencies | Cài đặt thư viện

Let's begin by installing the necessary Python libraries for our project.  
Bắt đầu bằng cách cài đặt các thư viện Python cần thiết cho project này.


In [ ]:
!pip install numpy pandas scikit-learn matplotlib

👉 Here, we import **all the necessary libraries** at once,  
so we don't have to add them again in later sections.

👉 Ở đây, ta **import tất cả các thư viện cần dùng** ngay từ đầu,  
để các phần phía sau không cần phải thêm nữa!


In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

## 📂 2. Load and Explore the Data  
*Tải và khám phá dữ liệu*

We'll start by loading the dataset.  
Chúng ta sẽ bắt đầu bằng cách tải bộ dữ liệu.

For this example, we assume that the dataset is available in CSV format.  
Trong ví dụ này, ta giả định dữ liệu có sẵn dưới dạng file `.csv`.

You can upload your own dataset, or use a sample dataset from a URL.  
Bạn có thể **tự upload file CSV** của mình, hoặc dùng **file mẫu có sẵn từ đường dẫn URL**.

📌 In this project, we use a pre-cleaned sample file: `spam.csv`  
📌 Trong project này, mình dùng một file mẫu đã được làm sạch: `spam.csv`


## ⬆️ Upload Dataset to Colab  
*Tải dữ liệu lên Colab*

Before we can use the dataset, we need to upload it into this Colab session.  
Trước tiên, ta cần **tải file dữ liệu (datasheet)** lên Colab để sử dụng.

👉 In this example, the file is available in the `data/` folder on GitHub.  
👉 Trong ví dụ này, file nằm trong thư mục `data/` của GitHub repo.

📥 You should **download the file to your local machine first**,  
then **upload it here** using the upload button below.  
📥 Bạn cần **tải file về máy trước**,  
rồi **upload lên Colab bằng nút phía dưới**.


In [10]:
# ⬆️ Upload the Data
# Tải dữ liệu lên Colab

from google.colab import files # Tải dữ liệu lên
import pandas as pd
uploaded = files.upload()
df = pd.read_csv("spam.csv") # kiểm tra dữ liệu sau khi tải lên
print(df)  # Xem thử 5 dòng đầu


Saving spam.csv to spam.csv
     label                                        message
0      ham           Please review the attached document.
1      ham        Don't forget about the meeting at 3 PM.
2      ham                      I'll call you back later.
3      ham       How was your weekend? Did you go hiking?
4      ham                      I'll call you back later.
...    ...                                            ...
9995   ham  Let's grab lunch tomorrow at our usual place.
9996   ham                      I'll call you back later.
9997   ham                      I'll call you back later.
9998   ham                      See you at the gym later!
9999   ham                      I'll call you back later.

[10000 rows x 2 columns]


## 🔄 Load Data and Convert Labels to Numbers  
*Load dữ liệu và chuyển đổi nhãn thành số*

Next, we will load the data and convert the labels (Spam/Not Spam) into numeric values.  
Tiếp theo, chúng ta sẽ tải dữ liệu và chuyển các nhãn (Spam/Không Spam) thành giá trị số.

### 🚀 Convert Labels to Numeric | Chuyển đổi nhãn thành số:

We can map **Spam** to 1 and **Not Spam** to 0.  
Chúng ta sẽ ánh xạ **Spam** thành 1 và **Không Spam** thành 0.




In [ ]:
# 1️⃣ Load dữ liệu
df = pd.read_csv("spam.csv", encoding="latin-1")[['label', 'message']]
df.columns = ['label', 'message']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})  # Chuyển đổi nhãn thành số

## 🧑‍💻 3. Preprocessing Data  
*Tiền xử lý dữ liệu*

Now, let's preprocess the data by handling missing values, encoding categorical features, and splitting the data into training and testing sets.  
Bây giờ, chúng ta sẽ tiền xử lý dữ liệu bằng cách xử lý các giá trị thiếu, mã hóa các đặc trưng phân loại và chia dữ liệu thành bộ huấn luyện và bộ kiểm tra.

### 🚀 1. Handle Missing Values | Xử lý giá trị thiếu:
Before we continue, let's check if there are any missing values in the dataset.  
Trước khi tiếp tục, chúng ta sẽ kiểm tra xem có giá trị nào bị thiếu trong dữ liệu không.


In [ ]:
# Check for missing values
print(df.isnull().sum())  # Kiểm tra số lượng giá trị thiếu trong mỗi cột

# 2️⃣ Tiền xử lý văn bản
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Xóa dấu câu
    text = re.sub(r"\d+", "", text)  # Xóa số
    return text

df['message'] = df['message'].apply(clean_text)

# 3️⃣ Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# 4️⃣ Chuyển đổi văn bản thành vector số
vectorizer = TfidfVectorizer()

# Bước 1: Huấn luyện vectorizer với dữ liệu huấn luyện
vectorizer.fit(X_train)

# Giả sử vectorizer là đối tượng TfidfVectorizer đã huấn luyện
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

# Bước 2: Chuyển đổi dữ liệu huấn luyện thành vector số
X_train_tfidf = vectorizer.transform(X_train)

X_test_tfidf = vectorizer.transform(X_test)

## 🏗️ 4. Build Logistic Regression Model  
*Xây dựng mô hình Logistic Regression*

Now, we will build the **Logistic Regression** model using **scikit-learn** and train it on the preprocessed data.  
Bây giờ, chúng ta sẽ xây dựng mô hình **Logistic Regression** sử dụng **scikit-learn** và huấn luyện mô hình với dữ liệu đã được tiền xử lý.

### 🚀 Train the Logistic Regression Model | Huấn luyện mô hình Logistic Regression:

We will initialize the logistic regression model and train it using the training data.  
Chúng ta sẽ khởi tạo mô hình logistic regression và huấn luyện nó với bộ dữ liệu huấn luyện.


In [ ]:
# 5️⃣ Huấn luyện mô hình Logistic Regression
model = LogisticRegression()

model.fit(X_train_tfidf, y_train)

## 📊 5. Evaluate the Model  
*Đánh giá mô hình*

After training the model, we need to evaluate its performance using metrics such as **accuracy**, **precision**, **recall**, and **F1-score**.  
Sau khi huấn luyện mô hình, chúng ta cần đánh giá hiệu suất của nó bằng các chỉ số như **độ chính xác**, **độ chính xác (precision)**, **độ nhạy (recall)** và **F1-score**.

### 🚀 Evaluate Model Performance | Đánh giá hiệu suất mô hình:

We will use **scikit-learn's metrics** to evaluate the model's performance on the test set.  
Chúng ta sẽ sử dụng các **chỉ số trong scikit-learn** để đánh giá hiệu suất của mô hình trên bộ kiểm tra.



In [ ]:
# 6️⃣ Đánh giá mô hình
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

## 💾 6. Save the Model  
*Lưu mô hình*

After training and evaluating the model, we can save it to disk so that it can be reused later without retraining. We will use the **joblib** library to save the model.  
Sau khi huấn luyện và đánh giá mô hình, chúng ta có thể lưu mô hình vào ổ đĩa để có thể tái sử dụng sau mà không cần huấn luyện lại. Chúng ta sẽ sử dụng thư viện **joblib** để lưu mô hình.



In [ ]:
# Lưu mô hình LogisticRegression đã huấn luyện
joblib.dump(model, "phanloaiemail.pkl")
print("Mô hình đã được lưu!")



# Full Code


In [11]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1️⃣ Load dữ liệu
df = pd.read_csv("spam.csv", encoding="latin-1")[['label', 'message']]
df.columns = ['label', 'message']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})  # Chuyển đổi nhãn thành số

# 2️⃣ Tiền xử lý văn bản
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Xóa dấu câu
    text = re.sub(r"\d+", "", text)  # Xóa số
    return text

df['message'] = df['message'].apply(clean_text)

# 3️⃣ Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# 4️⃣ Chuyển đổi văn bản thành vector số
vectorizer = TfidfVectorizer()

# Bước 1: Huấn luyện vectorizer với dữ liệu huấn luyện
vectorizer.fit(X_train)

# Giả sử vectorizer là đối tượng TfidfVectorizer đã huấn luyện
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

# Bước 2: Chuyển đổi dữ liệu huấn luyện thành vector số
X_train_tfidf = vectorizer.transform(X_train)

X_test_tfidf = vectorizer.transform(X_test)

# 5️⃣ Huấn luyện mô hình Logistic Regression
model = LogisticRegression()

model.fit(X_train_tfidf, y_train)

# 6️⃣ Đánh giá mô hình
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Lưu mô hình LogisticRegression đã huấn luyện
joblib.dump(model, "phanloaiemail.pkl")
print("Mô hình đã được lưu!")



Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1691
           1       1.00      1.00      1.00       309

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

Mô hình đã được lưu!


#**Next, let's build the interface using Streamlit.**

*Tiếp theo, ta sẽ xây dựng giao diện sử dụng Streamlit.*


#First, we need to install the required libraries!
Đầu tiên, ta cần cài các thư viện cần thiết!

In [4]:
# streamlit
!pip install streamlit

# cai dat pyngrok
!pip install pyngrok



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00


#**Next, let's create the interface.**
*Sau đó, ta sẽ tạo giao diện.*


In [12]:
%%writefile app.py
import streamlit as st
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Tải mô hình đã huấn luyện
model = joblib.load("phanloaiemail.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Tiêu đề ứng dụng
st.title("📩 Phân loại Email: Spam hay Không Spam?")

# Hướng dẫn sử dụng
st.write("Nhập nội dung email vào ô bên dưới để kiểm tra liệu đó có phải là spam không.")

# Nhập nội dung email
email_text = st.text_area("Nhập nội dung email:", "")

# Dự đoán
if st.button("📌 Phân loại Email"):
    if email_text.strip() == "":
        st.warning("Vui lòng nhập nội dung email để phân loại.")
    else:
        # Tiền xử lý và chuyển đổi văn bản thành vector
        email_tfidf = vectorizer.transform([email_text])
        prediction = model.predict(email_tfidf)[0]

        # Hiển thị kết quả
        if prediction == 1:
            st.error("🚨 Đây có thể là một email spam!")
        else:
            st.success("✅ Đây có vẻ là một email hợp lệ.")



Overwriting app.py


# At this point, to run the app, you need to log in to ngrok and get the authtoken!
Đến đây, để chạy App, bạn cần đăng nhập trên ngrok, rồi lấy authtoken!


- Step 1: Create an account at: https://dashboard.ngrok.com/login?state=0MZdko9buyFsE2RW-1tFrA4KegSYNVU7JMBTL5MJA_luBJvdgi7y3HJkDhj2KUskjAWLa-ObG_Z3DNV1e5I_xlhHktGPw2fSzQUKaPEQCB_vp1wg2WkJm-EBhSPLH7rsgqjDp-b8c1tJz6qnzC7zmV7yYzgdyXSBmHRb6RUnDXzheBwPCFyFgGy-

- Step 2: Get your authtoken here: https://dashboard.ngrok.com/get-started/your-authtoken

In [5]:
# Cần đăng nhập trên ngrok để lấy autoken! (Need to login on ngrok to get autoken!)
# Đây là autoken của tôi, bạn cần tự lấy autoken thay vào! (This is my autoken, you have to auto get autoken!)
!ngrok authtoken "2vnX9rYN7CO6Ah7xA73xsu2HF1R_7JeDiWoAKyKFJHPXc2BiX"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


#**Next, run the code below, and click on the link to go to our web app.**
*Sau đó, ta chạy đoạn code bên dưới, click vào link để chuyển đến web của chúng ta.*


In [13]:
#Chạy App
from pyngrok import ngrok
public_url = ngrok.connect(8501, "http")  # Xóa tham số "port="
print(f"Truy cập Streamlit tại: {public_url}")

!streamlit run app.py


Truy cập Streamlit tại: NgrokTunnel: "https://963d-35-223-50-136.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.223.50.136:8501

  Stopping...
  Stopping...
Exception ignored in atexit callback: <function _exit_function at 0x7a9a04228d60>
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/util.py", line 323, in _exit_function
    def _exit_function(info=info, debug=debug, _run_finalizers=_run_finalizers,
    
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/server.py", line 469, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 324, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventlo